In [1]:
import scvi
import scanpy as sc
import os
import pandas as pd
import numpy as np
from cfp.external import CFJaxSCVI
from cfp.metrics import compute_metrics

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/optuna/study/_optimize.py:29: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
split = 5
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex"
adata_train = sc.read(os.path.join(output_dir, f"adata_train_{split}.h5ad"))
adata_test = sc.read(os.path.join(output_dir, f"adata_test_{split}.h5ad"))
adata_ood = sc.read(os.path.join(output_dir, f"adata_ood_{split}.h5ad")) 




In [3]:
vae = CFJaxSCVI.load("/lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/10_2048_5", adata=adata_train)

INFO     File /lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/10_2048_5/model.pt already         
         downloaded                                                                                                


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/model/base/_utils.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be a

INFO     Jax module moved to cuda:0.Note: Pytorch lightning will show GPU is not being used for the Trainer.       


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU availabl

Epoch 1/18:   0%|          | 0/18 [00:00<?, ?it/s]

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:385: You have overridden `transfer_batch_to_device` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.


Epoch 1/18:   6%|▌         | 1/18 [00:02<00:48,  2.85s/it, v_num=1, train_loss_step=405, train_loss_epoch=405]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 1/18:   6%|▌         | 1/18 [00:02<00:49,  2.90s/it, v_num=1, train_loss_step=405, train_loss_epoch=405]


In [4]:
adata_test.obsm["X_scVI"] = vae.get_latent_representation(adata_test)
adata_ood.obsm["X_scVI"] = vae.get_latent_representation(adata_ood)



INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [5]:
adata_test.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_test)
adata_ood.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_ood)

In [9]:
metrics_ood = {}
for cond in adata_ood.obs["condition"].unique():
    if "Vehicle" in cond:
        continue
    true_dist = adata_ood[adata_ood.obs["condition"]==cond].X.toarray()
    pred_dist = adata_ood[adata_ood.obs["condition"]==cond].obsm["reconstruction"]
    metrics_ood[cond] = compute_metrics(true_dist, pred_dist)

In [7]:
out_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/reconstruction"

In [11]:
pd.DataFrame.from_dict(metrics_ood).to_csv(os.path.join(out_dir, "reconstruction_vae.csv"))

In [6]:
metrics_test = {}
for cond in adata_test.obs["condition"].unique():
    if "Vehicle" in cond:
        continue
    true_dist = adata_test[adata_test.obs["condition"]==cond].X.toarray()
    pred_dist = adata_test[adata_test.obs["condition"]==cond].obsm["reconstruction"]
    metrics_test[cond] = compute_metrics(true_dist, pred_dist)

In [9]:
pd.DataFrame.from_dict(metrics_test).to_csv(os.path.join(out_dir, "reconstruction_vae_test.csv"))

In [6]:
fraction_negative = {}
for cond in adata_ood.obs["condition"].unique():
    pred_dist = adata_ood[adata_ood.obs["condition"]==cond].obsm["reconstruction"]
    neg = (pred_dist < 0).sum().sum()
    fraction_negative[cond] = [neg / ((pred_dist.shape[0]) * (pred_dist.shape[1]))]

In [8]:
pd.DataFrame.from_dict(fraction_negative).to_csv(os.path.join(out_dir, "negativity_vae.csv"))